In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchsample as ts


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

ModuleNotFoundError: No module named 'torchsample'

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data',train=True,
                                       download=True,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True,num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data',train=False,
                                       download=True,transform=transform)
testloader = torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True,num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 4, 1, 2)
        self.conv2 = nn.Conv2d(64, 64, 4, 1, 2)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.batch = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.drop = nn.Dropout2d(p=0.2)
        self.fc1 = nn.Linear(1024, 500) #Potential err in dimension
        self.fc2 = nn.Linear(500, 500)
        self.fc3 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.batch(self.conv1(x))) #conv1
        x = self.pool(F.relu(self.conv2(x)))  #conv2
        x = self.drop(x)
        x = F.relu(self.batch(self.conv2(x))) #conv3
        x = self.pool(F.relu(self.conv2(x)))  #conv4
        x = self.drop(x)
        x = F.relu(self.batch(self.conv2(x))) #conv5
        x = self.drop(F.relu(self.conv3(x)))  #conv6
        x = F.relu(self.batch(self.conv3(x))) #conv7
        x = F.relu(self.batch(self.conv3(x))) #conv8
        x = self.drop(x)
        # Fully connected
        x = x.view(-1, self.num_flat_features(x)) #Potential err in dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
net.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = Variable(inputs).cuda(),Variable(labels).cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print('Finished Training')

In [ ]:
correct = 0
total = 0
# with torch.no_grad():
for data in testloader:
    images, labels = data
    images, labels = Variable(images).cuda(),Variable(labels).cuda()
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))